In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
from unidecode import unidecode

In [38]:
categories = ['aabp', 'aabd', 'aaba', 'aabas', 'aabsa', 'aabsas']
csv_rows = {category: [] for category in categories}


for year in range(1974, 2023):
    page = requests.get(f"https://www.sportsoddshistory.com/aa-awd/?y={year}")
    soup = BeautifulSoup(page.text, "html.parser")
    
    # Some year don't have historical odds
    if soup.find('a', {'id': 'aabp'}) is None:
        continue
    
    for category in categories:
        try:
            elem_rows = soup.find('a', {'id': category}).find_next_sibling('table').find('tbody').find_all('tr')
            for elem_row in elem_rows:
                # First column is year
                data_row = [year]
                
                # Append the rest of the columns
                data_row.extend([unidecode(td_elem.text).strip() for td_elem in elem_row.find_all('td')])
                
                # Convert last column (winner)
                data_row[-1] = True if data_row[-1] == "**  WINNER  **" else False
                
                # Convert second last column (odds)
                data_row[-2] = american_to_decimal_odds(data_row[-2])
                csv_rows[category].append(data_row)
            
        except Exception as err:
            print(f'Error: Year {year}, category {category}, description {err}')
    
# Print to files
for category in categories:
    with open(f'data/archived_odds/{category}.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_rows[category])
        

Error: Year 1975, category aabd, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1983, category aabd, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1988, category aabd, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1988, category aabsa, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1988, category aabsas, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1989, category aabd, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1989, category aabsa, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1989, category aabsas, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1990, category aabd, description 'NoneType' object has no attribute 'find_next_sibling'
Error: Year 1990, category aabsa, description 'NoneType' object has no attribute 'find_next_si

In [37]:
def american_to_decimal_odds(odds):
    if odds == 'N/A':
        return 0
    if odds[0] == '-':
        return (100 + int(odds[1:])) / int(odds[1:])
    return (100 + int(odds[1:])) / 100

def american_to_implied_prob(odds):
    return 1 / american_to_decimal_odds(odds)